__Complete all sub-tasks marked with ## TO DO! ## and submit the filled notebook on OLAT__ \
__Using a GPU is recommended here__

### Transfer Learning ###
Aim of this notebook is to implement the concept of transfer learning to train a bigger dataset. We try to compete on a well-known competiton on Kaggle known as Dog Breeds Identification. Read more about it here:

https://www.kaggle.com/c/dog-breed-identification/overview



To train a model on the Dog breeds dataset using transfer learning and submit your results to Kaggle.
Note: Below notebook gives some tips to run the code in pytorch. 

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import shutil




In [3]:
use_cuda = torch.cuda.is_available()
print(use_cuda) # This implies Cuda is available for work 

True


In [4]:
from AlexNet import AlexNet
from train_test import start_train_test

In [ ]:
####################################################################################################
## TO DO! : Register on Kaggle With Your respective GroupName  (For example: WS20_VDL_GROUP_01)    ##
####################################################################################################

In [ ]:
####################################################################################################
## TO DO! : Download the Dog-Breeds dataset in folder "data"                                      ##
## from the Kaggle competition link mentioned above                                               ##
####################################################################################################

In [5]:
####################################################################################################
## TO DO! : Make your dataset to and dataloaders for the  test data 
####################################################################################################
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
import transform as trf
from pathlib import Path

train_source_dir='Data_kaggle/train/'
train_dest_dir='data/train/'
test_source_dir='Data_kaggle/test/'
test_dest_dir='data/test/'

In [6]:
####################################################################################################
## TO DO! : Split train data into 20% validation set and make dataloaders for train and val split ##
####################################################################################################

#splitting the data into training and validation using the training data we have just taken out as we already have the test data

# we are using the transform methods provided in the imports and the same automatically uses the augmentation internally
train_data = datasets.ImageFolder(train_dest_dir,       
                    transform=trf.transform_training())

#test_data = datasets.ImageFolder()

#splitting the dataset into training and validation set but I am not sure if this is the place to do it ...positioning wise after transform and before dataloaders

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(train_data, [train_size, val_size])

print(len(train_dataset))
print(len(validation_dataset))


# train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=256,  #The batch size is the same as what we have in the config.py
#                                shuffle = True)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256,  #The batch size is the same as what we have in the config.py
                               shuffle = True)

validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=256,  #The batch size is the same as what we have in the config.py
                               shuffle = True)

# print(train_data[0])
dataiter = iter(train_dataloader)
data,labels = dataiter.next()

print(type(data))
print(data.shape)
print(labels.shape)
#data.shape gives the output of the form [batch_size,colour,height,width]



8177
2045
<class 'torch.Tensor'>
torch.Size([256, 3, 224, 224])
torch.Size([256])


In [ ]:
# HINT: 
# One can make their own custom dataset and dataloaders using the CSV file or
# Convert the Dog-breed training dataset into Imagenet Format, where all images of one class are in a
# folder named with class as in the below given format. Standard Pytorch Datasets and Dataloaders can then be applied
# over them
# Root
# |
# |---Class1 ___Img1.png
# |          ___Img2.png
# |
# |---Class2 ___Img3.png
# |          ___Img4.png
# |....
# |....

#Custom Dataset
#loading and sorting the labels based on the id column 
# The data has been sorted to get the labels in sync with the ids specified in the labels,csv

labels = pd.read_csv('Data_kaggle/labels.csv')
labels.sort_values(by=['id']) 


# Arranging the data in an appropriate dataset of classes
unique_labels=list(labels.breed.unique())
count=0
# getting the list of ids and copy pasting them in appropriate folders
for x in unique_labels:
#      print("===========================================================\n")
    id_list_by_class=list(labels.query('breed==@x')['id'])
    count+=1
    class_path=os.path.join(train_dest_dir,str(x)) 
    os.makedirs(class_path,exist_ok=True)
    for img in id_list_by_class:
        src_file=os.path.join(train_source_dir,img+'.jpg')
        shutil.copy2(src_file, class_path) 
print("class folders along with the appropriate images done successfully")


__Train famous Alexnet model on Dog breeds dataset. It is not easy to train the alexnet model from 
scratch on the Dog breeds data itself. Curious minds can try for once to train Alexnet from scratch. We adopt Transfer Learning here. We 
obtain a pretrained Alexnet model trained on Imagenet and apply transfer learning to it to get better results.__

## Transfer Learning

In [7]:
####################################################################################################
## TO DO! :  Freeze the weigths of the pretrained alexnet model and change the last classification layer
##from 1000 classes of Imagenet to 120 classes of Dog Breeds, only classification layer should be 
## unfreezed and trainable                                                                        ##
####################################################################################################
from torch import optim #importing for Adam Optimizer as defined in the config file 

import torchvision.models as models
pretrained_alexnet = models.alexnet(pretrained=True)

#printing the model
# print(pretrained_alexnet)


#freezing the alexnet learned parameters except the last classification layer 

for param in pretrained_alexnet.parameters():
    param.requires_grad = False
    
#adding the final layer for classification

print("==========================================================")
pretrained_alexnet.classifier[6]= nn.Sequential(
                                  nn.Linear(4096, 1000), 
                                  nn.ReLU(), 
                                  nn.Dropout(0.2),
                                  nn.Linear(1000, 120))

#pretrained_alexnet.classifier[6] = nn.Linear(4096,120)


print(pretrained_alexnet)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrained_alexnet = pretrained_alexnet.to(device)  #moving thr alexnet to gpu 

# # for param in pretrained_alexnet.classification.parameters():
# #     param.requires_grad = True

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(pretrained_alexnet.parameters(), lr=0.01)


# print(pretrained_alexnet)
# # Below function will directly train your network with the given parameters to 5 epochs
# # You are also free to use function learned in task 1 to train your model here 
# # train_loss, test_loss = start_train_test(pretrained_alexnet, trainloader, testloader, criterion)

#learning rate was changed to 0.001 to achieve better accuracy
train_loss, test_loss = start_train_test(pretrained_alexnet, train_dataloader, validation_dataloader, criterion)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

C:\Users\ASUS\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "



=> Training Epoch #1, LR=0.0010
| Epoch [  1/  5] 		Loss: 3.4090 Acc@1: 15.690%

| Validation Epoch #1			Loss: 3.2679 Acc@1: 26.36%
* Test results : Acc@1 = 26.36%
| Elapsed time : 0:01:04

=> Training Epoch #2, LR=0.0010
| Epoch [  2/  5] 		Loss: 3.0795 Acc@1: 24.985%

| Validation Epoch #2			Loss: 3.0357 Acc@1: 28.26%
* Test results : Acc@1 = 28.26%
| Elapsed time : 0:02:00

=> Training Epoch #3, LR=0.0010
| Epoch [  3/  5] 		Loss: 3.1342 Acc@1: 27.015%

| Validation Epoch #3			Loss: 3.0723 Acc@1: 27.63%
* Test results : Acc@1 = 28.26%
| Elapsed time : 0:02:57

=> Training Epoch #4, LR=0.0010
| Epoch [  4/  5] 		Loss: 2.7383 Acc@1: 29.179%

| Validation Epoch #4			Loss: 2.7522 Acc@1: 30.17%
* Test results : Acc@1 = 30.17%
| Elapsed time : 0:03:53

=> Training Epoch #5, LR=0.0010
| Epoch [  5/  5] 		Loss: 2.7591 Acc@1: 30.207%

| Validation Epoch #5			Loss: 2.7276 Acc@1: 30.42%
* Test results : Acc@1 = 30.42%
| Elapsed time : 0:04:50


## Making Kaggle Submission

In [8]:
from transform import transform_testing
import PIL.Image
import torch.nn.functional as F
import numpy as np

In [9]:
### Not So optimal Code: This can take upto 2 minutes to run: You are free to make an optimal version :) ###
# It iterates over all test images to compute the softmax probablities from the last layer of the network
augment_image = transform_testing()
test_data_root = 'Data_kaggle/test/' 
test_image_list = os.listdir(test_data_root) # list of test files 
result = []
for img_name in test_image_list:
    img = PIL.Image.open(test_data_root + img_name)
    img_tensor = augment_image(img)
    with torch.no_grad():
        #output = pretrained_resnet(img_tensor.unsqueeze_(0).cuda())
        output = pretrained_alexnet(img_tensor.unsqueeze_(0).cuda())
        probs = F.softmax(output, dim=1)
    result.append(probs.cpu().numpy())
all_predictions = np.concatenate(result)
print(all_predictions.shape)

(10357, 120)


In [10]:
df = pd.DataFrame(all_predictions)
file_list = os.listdir('data/train') # list of classes to be provided here
df.columns = sorted(file_list)

# insert clean ids - without folder prefix and .jpg suffix - of images as first column
test_data_root = 'Data_kaggle/test/' # list of all test files here
test_image_list = os.listdir(test_data_root)
df.insert(0, "id", [e[:-4] for e in test_image_list])
df.to_csv(f"sub_1_alexnet.csv", index=False)

### TO DO!: ###
Submit the created CSV file to Kaggle, with a score(cross entropy loss) not more than __2.0__\
Take a snapshot of your evaluation score and  include the image here ...
For example :
![title](Kaggle_submission.png)